### Importing Packages

In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import random

In [2]:
# data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/gaussian_large_d_1.tex')
# data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/gaussian_small_d_1.tex')
# data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/uniform_large_d_1.tex')
# data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/uniform_small_d_1.tex')
data = pd.read_csv('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Sanity Checks/Demos/stacked_all.csv')
# data = pd.read_csv('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Sanity Checks/Demos/gaussmall.csv', header=None)
# data = pd.read_csv('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Sanity Checks/Demos/unismall.csv', header=None)
array = np.array(data)
df = pd.DataFrame(array)

# df = pd.read_excel('/Users/elleemortensen/Downloads/D3Softball.xlsx', index_col=[0])
# df.replace('---', np.nan, inplace=True)
# df.dropna(inplace=True)

In [ ]:
# for i in range(25):
#     df.iloc[:,i] = df.iloc[:,i].round()
#     df.iloc[:,i] = df.iloc[:,i].astype(int)
#     df.iloc[:,i] = df.iloc[:,i].astype("category")
# df.iloc[:,150] = df.iloc[:,150].astype("category")

### Splitting Data

In [ ]:
# split the dataset into training and testing data
# test_size: this is the percentage of data used for testing (20% in this case), so the rest is used for training data (80% in this case)
# random_state: this is a random number chosen that should be used each time to ensure we get the same data split each time
# X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size = 0.2, random_state = 42)

### XGBoost

In [ ]:
# columns_to_convert = [
#     'G', 'AB', 'H (Offensive)', 'H/G', 'BA', '2B (Offensive)', '2B/G',
#     '3B (Offensive)', '3B/G', 'Innings Pitched', 'K (Def)', 'K/G (Def)',
#     'BB Allowed', 'BB/G (Def)', 'K/BB', 'HA', 'HA/G', 'Runs Allowed',
#     'Runs Allowed/G', 'ER Allowed', 'ERA', 'WHIP', 'PO', 'A', 'E', 'E/G',
#     'FPCT', 'HBP (Offensive)', 'HBP/G', 'BB (Offensive)', 'BB/G (Off)',
#     'SF (Offensive)', 'SH (Offensive)', 'OBP', 'SHO', 'SHO %', 'SB',
#     'SB/G', 'TB', 'TB/G', 'SLG PCT', 'R', 'R/G', 'DP', 'DP/G',
#     'K (Off)', 'K/G (Off)'
# ]

# # Convert the columns to float
# df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [ ]:
# create model instance
# n_estimators: number of trees(estimators) the model uses --> the more used, the more accurate the model is
# max_depth: maximum depth of tree --> higher number makes model more complex, but too high can cause overfitting
# learning_rate: quantifies each tree's contribution to total prediction --> lower number takes longer, but can lead to better generalization
# objective: binary:logistic outputs probabilities. if classification is wanted, use binary:hinge
# bst = XGBClassifier(n_estimators = 2, max_depth = 2, learning_rate = 1, objective = 'binary:logistic', enable_categorical = True)

# # fit model with the training data
# bst.fit(X_train, y_train)

# # make predictions for the test dataset
# preds = bst.predict(X_test)

# # print predictions
# print(preds)

# # print model Accuracy (how often the classifier is correct)
# print("Accuracy:", metrics.accuracy_score(y_test, preds))

# Create XGBoost model instance
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic', enable_categorical=True)

# Fit model with the training data
bst.fit(X_train_corner, y_train_corner)

# Make predictions for the test dataset
preds = bst.predict(X_test_corner)

# Print predictions
print("Predictions:", preds)

# Calculate and print F1 score
f1 = metrics.f1_score(y_test_corner, preds)
print("F1 Score:", f1)

### FIC

In [11]:
class FeatureClassifier:
  def __init__(self,reqAcc = 0.01, classifier = 'DecisionTree', bias = [], control = None, n_jobs = None, random_state = 42):
    self.featureClassifiers=[] #list of all the classifiers of all the selected features
    self.reqAcc=reqAcc #user specified cutoff value
    self.indexLs=[] # list of mapped index values to featureClassifiers
    self.flag=0
    self.bias=bias # list of biases for each and every label
    self.control=control #overfitting control for decision trees
    self.classifier=classifier #the classifier which is preferred by the user
    self.dic={'DecisionTree':0,'LinearRegression':1,'SVM':2,'LogisticRegression':3} #a dictionary which maps the classifier to its index
    self.n_jobs=n_jobs
    self.random_state=random_state
    self.num_lables = None

  def finIndex(self):
    #finds the index where the reqAcc condition fails and also created the indexLs[] for mapping
    for i in range(len(self.featureClassifiers)):
      if self.featureClassifiers[i][1] < self.reqAcc:
        return i
      self.indexLs.append(self.featureClassifiers[i][2])
    self.flag=1
    return i
  def fit(self,x,y):
    #applied the model to the dataset. The model is trained and saved for further prediction
    self.num_lables=len(set(y.flatten()))
    bestfeatures = SelectKBest(score_func=chi2,k=1)
    print(x,y)
    fit = bestfeatures.fit(x,y)

    for i in range(len(x[0])):
      clf=[DecisionTreeClassifier(max_depth=self.control,random_state=self.random_state),LinearRegression(n_jobs=self.n_jobs),SVC(gamma=self.control,random_state=self.random_state), LogisticRegression(penalty=self.control,random_state=self.random_state)][self.dic[self.classifier]]
      X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33,random_state=self.random_state)
      clf.fit(X_train[:,i:i+1],y_train)
      self.featureClassifiers.append((clf,fit.scores_[i],i))
    self.featureClassifiers.sort(key=lambda x:x[1],reverse=True)
    index=self.finIndex()
    if self.flag==0:
      self.featureClassifiers=self.featureClassifiers[:index]
    return

  def predict(self,x):
    #given a list of inputs, predicts the possible outputs
    if not self.bias:
      self.bias=np.zeros(self.num_lables)
    if len(self.bias)<self.num_lables:
      raise AttributeError('Please check the lenth of bias list')
    yPred=[]
    for i in range(len(x)):
      pred_arr=np.zeros(self.num_lables)
      for j in range(len(self.indexLs)):
        pred=np.round(self.featureClassifiers[j][0].predict([[x[i][self.indexLs[j]]]]))
        pred_arr[pred]+=self.featureClassifiers[j][1]+self.bias[pred[0]]
      yPred.append(np.argmax(pred_arr))
    return yPred

In [ ]:
#train the model using the training sets
clf1=FeatureClassifier(0, classifier='DecisionTree', control=3, random_state=42)
# clf1.fit(X_train,y_train.reshape(-1,1))
clf1.fit(np.array(X_train), np.array(y_train)[:,np.newaxis].astype(int))

#predict the response for the test dataset
#model accuracy (how often the classifier is correct)
print("Accuracy:", metrics.accuracy_score(np.array(y_test).astype(int),clf1.predict(np.array(X_test))))

print(clf1.indexLs)
clf1.featureClassifiers
print(clf1.indexLs)

In [13]:
# Assuming FeatureClassifier is correctly implemented for DecisionTreeClassifier
clf1 = FeatureClassifier(0, classifier='DecisionTree', control=3, random_state=42)

# Fit model with the training data
clf1.fit(np.array(X_train_corner), np.array(y_train_corner).astype(int))

# Predict the response for the test dataset
y_pred = clf1.predict(np.array(X_test_corner))

# Calculate and print F1 score
f1 = metrics.f1_score(np.array(y_test_corner).astype(int), y_pred)
print("F1 Score:", f1)

# Print other relevant outputs for debugging
print(clf1.indexLs)

[[ 1.91084413  0.7977536   3.25609559 ...  0.          0.
   1.        ]
 [ 0.61496164  1.30631956 -0.15622361 ...  1.          1.
   0.        ]
 [-0.07089384  0.84888389  0.02390265 ...  0.          1.
   0.        ]
 ...
 [ 0.37466714  3.64690773  2.03829623 ...  2.          1.
   0.        ]
 [ 0.02271092  0.84125244  1.17746719 ...  0.          0.
   2.        ]
 [ 1.04994748  0.38292255  1.17680361 ...  0.          0.
   1.        ]] [1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0 0
 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1
 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1
 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1
 1 0 0 0 0 1 0 1 1 1 1 0 1 0 0 0 0 1 0 0]


ValueError: Input X must be non-negative.

In [17]:
print(sum(y_test_corner<0))

0


In [ ]:
# Identify columns with any negative values
columns_with_negatives = df.columns[(df < 0).any()]

# Drop those columns
df = df.drop(columns=columns_with_negatives)
df.info

### KNN

In [ ]:
#create a knn classifier
#n_neighbors: predicting the label of the data point by looking at the 3 closest data points and getting them to "vote"
#algorithm: we may need to look at this if it misbehaves
neigh = KNeighborsClassifier(n_neighbors=5)

#train the model using the training sets
neigh.fit(X_train, y_train)

#predict the response for the test dataset
y_pred = neigh.predict(X_test)

#print predictions
print(y_pred)

#model accuracy (how often the classifier is correct)
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))

In [ ]:
# Create KNN classifier instance
neigh = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
neigh.fit(X_train_corner, y_train_corner)

# Predict the response for the test dataset
y_pred = neigh.predict(X_test_corner)

# Print predictions
print("Predictions:", y_pred)

# Calculate and print F1 score
f1 = metrics.f1_score(y_test_corner, y_pred)
print("F1 Score:", f1)

### SVM

#### SVM Accuracy

In [ ]:
#Create a svm Classifier
# kernel: options for kernel include linear, poly, rbf, sigmoid
    # linear: use this when data can be split by a linear function
    # poly (polynomial): use this when data can be split by a polynomial function
    # rbf (radial basis function): use this when there are clusters of one class inside another
    # sigmoid: use this when the split between classes is curved and irregular
clf = svm.SVC(kernel='linear')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# print predictions
print("Predictions:", y_pred)

# print model Accuracy (how often the classifier is correct)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

#### SVM F1 Scores

In [5]:
# Stacked
X_train_corner = df.iloc[:168, :23]
X_test_corner = df.iloc[168:241, :23]
y_train_corner =  df.iloc[:168, 24]
y_test_corner = df.iloc[168:241, 24]

# Gaussian
# X_train_corner = df.iloc[:120, :12]
# X_test_corner = df.iloc[120:, :12]
# y_train_corner =  df.iloc[:120, 12]
# y_test_corner = df.iloc[120:, 12]

# Uniform
# X_train_corner = df.iloc[:240, :24]
# X_test_corner = df.iloc[240:, :24]
# y_train_corner =  df.iloc[:240, 24]
# y_test_corner = df.iloc[240:, 24]

In [ ]:
# Create SVM classifier instance with linear kernel
clf = svm.SVC(kernel='linear', random_state = 42)

# Train the model using the training sets
clf.fit(X_train_corner.iloc[:,0:], y_train_corner)

# Predict the response for test dataset
y_pred = clf.predict(X_test_corner.iloc[:,0:])

# Print predictions
print("Predictions:", y_pred)

# Calculate F1 score
print("F1 Score:", metrics.f1_score(y_test_corner, y_pred))